In [13]:
import numpy as np
import pandas as pd
import os

In [14]:
!pip install keras_tuner

  Using cached keras_tuner-1.4.7-py3-none-any.whl.metadata (5.4 kB)
  Using cached kt_legacy-1.0.5-py3-none-any.whl.metadata (221 bytes)
Using cached keras_tuner-1.4.7-py3-none-any.whl (129 kB)
Using cached kt_legacy-1.0.5-py3-none-any.whl (9.6 kB)



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
if 'google.colab' in str(get_ipython()):
    from google.colab import drive
    drive.mount('/content/drive')
    os.chdir('/content/drive/MyDrive')
else:
    path = r'C:/dev/machine_learning_project/deep_learning/classification/'
    os.chdir(path)

#Importing data

In [ ]:
dataframe = pd.read_excel('../../data/chiefs_knife_dataset.xlsx')
index_Ra = dataframe.columns.get_loc('Ra') # index of the surface roughness column for inserting the class. label

lower_specification_limit = 0.125 # lower bound of good quality product region
upper_specification_limit = 0.215  # upper bound of good quality product region

is_between_specification_bounds = (dataframe['Ra']>=lower_specification_limit) & (dataframe['Ra'] < upper_specification_limit)
good_product_range = np.where(is_between_specification_bounds,"good","bad")
dataframe.insert(index_Ra, 'Quality',good_product_range) # encoding: good quality product := 1 and poor quality product := 0

In [ ]:
dataframe

# constructing Features and Label

In [ ]:
X = dataframe.loc[:,'Original_Linienanzahl':'DFT_Median_sobel_Bereich'].values
y = dataframe['Quality'].values

In [ ]:
X

In [ ]:
y

#Encoding categorical data

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(y)

In [ ]:
y

In [ ]:
dataframe['Quality'] = y
dataframe

#Splitting dataset into training and test set

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [ ]:
test_size = 0.15
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size,shuffle=True, random_state=1)

#Scaling the features

In [ ]:
sc = MinMaxScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
X_train

#Hyperparameter definition

In [ ]:
# import keras_tuner as kt
# from keras_tuner import HyperParameters

# neurons_max = 1024
# neurons_min = 32
# neurons_step = 16
# dropout_min = 0.0
# dropout_max = 0.5
# dropout_step = 0.1
# l2_min = 0.0
# l2_max = 0.1
# l2_step = 0.01
# learning_rates = [1e-2, 1e-3, 1e-4,1e-5]
# activations = ['sigmoid', 'tanh']

#Model definition

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LeakyReLU, ELU, Activation
from tensorflow.keras.optimizers import Adam, SGD, Nadam, RMSprop
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
import keras_tuner as kt


def set_activation(activation_name):
    """
  set the activation function for the model
  """
    if activation_name == 'leaky_relu':
        activation = tf.keras.layers.LeakyReLU()
    else:
        activation = tf.keras.layers.Activation(activation_name)
    return activation


class DeepClassifierModel:
    hp = kt.HyperParameters()

    def __init__(self, input_dim):
        """
      initialize the hyperparameters for hyperparameter tuning
      """
        self.input_dim = input_dim
        self.neurons_max = 1024
        self.neurons_min = 32
        self.neurons_step = 16
        self.learning_rates = [1e-2, 1e-3, 1e-4, 1e-5]
        self.activation_name = ['relu', 'tanh', 'selu', 'leaky_relu']
        self.optimizer_name = ['adam', 'nadam', 'rmsprop', 'sgd']

    def build_model(self, hp):
        """
      create a NN architecture for the classifiction task
      hp: hyperparameter for hyperparameter tuning
      """
        model = tf.keras.models.Sequential()

        model.add(tf.keras.layers.Dense(
            units=hp.Int('hidden_1', min_value=self.neurons_min, max_value=self.neurons_max, step=self.neurons_step),
            activation=set_activation(hp.Choice('activation_1', values=self.activation_name)),
            input_dim=self.input_dim))

        model.add(tf.keras.layers.Dense(
            units=hp.Int('hidden_2', min_value=self.neurons_min, max_value=self.neurons_max, step=self.neurons_step),
            activation=set_activation(hp.Choice('activation_2', values=self.activation_name))))

        model.add(tf.keras.layers.Dense(
            units=hp.Int('hidden_3', min_value=self.neurons_min, max_value=self.neurons_max, step=self.neurons_step),
            activation=set_activation(hp.Choice('activation_3', values=self.activation_name))))
        
        model.add(tf.keras.layers.Dense(
            units=hp.Int('hidden_4', min_value=self.neurons_min, max_value=self.neurons_max, step=self.neurons_step),
            activation=set_activation(hp.Choice('activation_4', values=self.activation_name))))
        
        model.add(tf.keras.layers.Dense(
            units=hp.Int('hidden_5', min_value=self.neurons_min, max_value=self.neurons_max, step=self.neurons_step),
            activation=set_activation(hp.Choice('activation_5', values=self.activation_name))))

        model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

        model.compile(optimizer=self.set_optimizer(hp.Choice('optimizer', values=self.optimizer_name),hp.Choice('learning_rate', values=self.learning_rates)),
                      loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
        return model

    def set_optimizer(self, optimizer_name,learn_rate):
        """
      set the optimizer for the model
      """
        if optimizer_name == 'adam':
            return tf.keras.optimizers.Adam(learning_rate=learn_rate)
        elif optimizer_name == 'sgd':
            return tf.keras.optimizers.SGD(learning_rate=learn_rate)
        elif optimizer_name == 'nadam':
            return tf.keras.optimizers.Nadam(learning_rate=learn_rate)
        elif optimizer_name == 'rmsprop':
            return tf.keras.optimizers.RMSprop(learning_rate=learn_rate)

#random search for hyperparameter tuning

In [ ]:
classifier = DeepClassifierModel(X_train.shape[1])

tuner = kt.RandomSearch(
hypermodel=classifier.build_model,
objective='val_accuracy',
max_trials=60,
executions_per_trial=1,
overwrite=True,
directory="hyperparameter_tuning_classifier",
project_name="simple",
)

In [ ]:
# callback = EarlyStopping(monitor='val_loss', patience=5)
tuner.search(X_train, y_train, batch_size=16, epochs=15, validation_split=0.1)

In [ ]:
models = tuner.get_best_models(num_models=8)
models.results_summary()

In [ ]:
tuner.get_best_hyperparameters()[0].values

In [ ]:
# param = {'hidden_1': 288,
#  'activation_1': 'relu',
#  'hidden_2': 464,
#  'activation_2': 'leaky_relu',
#  'optimizer': 'rmsprop'}

In [ ]:
# history = models[0].fit(X_train, y_train, epochs=300, batch_size=8,validation_split=0.1)

In [ ]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense,LeakyReLU, ELU, Activation
# from tensorflow.keras.optimizers import Adam, SGD, Nadam, RMSprop
# from tensorflow.keras.callbacks import EarlyStopping
# import tensorflow as tf

# best_model = Sequential()

# best_model.add(Dense(units = 288, activation ='relu',input_dim=X_train.shape[1]))

# best_model.add(Dense(units =464, activation = 'leaky_relu'))

# best_model.add(Dense(units= 1, activation ='sigmoid'))

# best_model.compile(optimizer = RMSprop(learning_rate=0.001),loss='binary_crossentropy',metrics=['accuracy'])
# history = best_model.fit(X_train, y_train, epochs=100, batch_size=8,validation_split=0.1)

In [ ]:
# from sklearn.metrics import confusion_matrix, accuracy_score
# matrix_conf=confusion_matrix(y_test, best_model.predict(X_test).round())
# accuracy_model=accuracy_score(y_test, best_model.predict(X_test).round())
# print(f"{matrix_conf=}")
# print(f'{accuracy_model=}')

In [ ]:
# import matplotlib.pyplot as plt
# def show_loss_acc(h):
#   plt.figure(figsize=(12, 8))
#   plt.subplot(2, 2, 1)
#   plt.plot(h['loss'], label='train loss')
#   plt.plot(h['val_loss'], label='val loss')
#   plt.legend()
#   plt.subplot(2, 2, 2)
#   plt.plot(h['accuracy'], label='train accuracy')
#   plt.plot(h['val_accuracy'], label='val accuracy')
#   plt.legend()
#   plt.show()
# show_loss_acc(history.history)